In [8]:
import sqlite3
import json
from datetime import datetime
import sys
!{sys.executable} -m pip install dill

In [9]:
import dill

In [2]:
timeframe='2015-01'
sql_transaction=[]
start_row = 0
cleanup = 1000000

In [3]:
connection=sqlite3.connect('C:/Users/RakeshS/Videos/videos/ENGLISH MOVIES/RC_2015-01/{}.db'.format(timeframe))
c=connection.cursor()

In [4]:
def create_table():
    c.execute(""" CREATE TABLE IF NOT EXISTS parent_reply
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT,
    comment TEXT, subreddit TEXT, unix INT, score INT ) """)
    
def format_data(data):
    data= data.replace("\n"," newlinechar").replace("\r","newlinechar").replace('"',"'")
    return data

def find_existing_score(pid):
    try:
        sql="SELECT score FROM parent_reply WHERE parent_id='{}' LIMIT 1".format(pid)
        c.execute(sql)
        result=c.fetchone()
        if result!=None:
            return result[0]
        else:
            return False
    except Exception as e:
        #print("find_parent",e)
        return False

def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True
    
    

def find_parent(pid):
    try:
        sql="SELECT comment FROM parent_reply WHERE comment_id='{}' LIMIT 1".format(pid)
        c.execute(sql)
        result=c.fetchone()
        if result!=None:
            return result[0]
        else:
            return False
    except Exception as e:
        #print("find_parent",e)
        return False

    
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql=""" UPDATE parent_reply SET parent_id=?,comment_id=?,parent=?,comment=?,subreddit=?,time=?,unix=?,score=? WHERE parent_id=?;""".format(commentid, parentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print("s0-replace-comment insertion",str(e))
        
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0-has-parent insertion',str(e))
        
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0-no-parent insertion',str(e))

In [5]:
if __name__=='__main__':
    create_table()
    row_counter=0
    paired_rows=0

In [6]:
with open("C:/Users/RakeshS/Videos/videos/ENGLISH MOVIES/RC_2015-01/{}/RC_{}".format(timeframe.split('-')[0], timeframe), buffering=1000) as f:
    for row in f:
        row_counter+=1
        if row_counter > start_row:
            
            try:
                
                row=json.loads(row)
                parent_id=row['parent_id']
                body=format_data(row['body'])
                created_utc=row['created_utc']
                score=row['score']
                comment_id = row['name']
                subreddit=row['subreddit']
                parent_data=find_parent(parent_id)
                if score >= 2:
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if acceptable(body):
                                
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            
                else:
                    
                    if acceptable(body):
                        
                        if parent_data:
                            
                            sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            paired_rows += 1
                        else:
                            
                            sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
            except Exception as e:
                print(str(e))
                
        if row_counter % 100000 == 0:
            print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))
            
        if row_counter > start_row:
            
            if row_counter % cleanup == 0:
                
                print("Cleanin up!")
                sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                c.execute(sql)
                connection.commit()
                c.execute("VACUUM")
                connection.commit()
            
                    

Total Rows Read: 100000, Paired Rows: 5095, Time: 2019-11-28 14:24:03.763082
Total Rows Read: 200000, Paired Rows: 12121, Time: 2019-11-28 14:25:08.609877
Total Rows Read: 300000, Paired Rows: 19823, Time: 2019-11-28 14:26:11.498428
Total Rows Read: 400000, Paired Rows: 27702, Time: 2019-11-28 14:27:19.814826
Total Rows Read: 500000, Paired Rows: 34934, Time: 2019-11-28 14:28:29.343853
Total Rows Read: 600000, Paired Rows: 41293, Time: 2019-11-28 14:29:44.848723
Total Rows Read: 700000, Paired Rows: 47749, Time: 2019-11-28 14:31:01.828319
Total Rows Read: 800000, Paired Rows: 54552, Time: 2019-11-28 14:32:17.313390
Total Rows Read: 900000, Paired Rows: 62133, Time: 2019-11-28 14:33:26.047882
Total Rows Read: 1000000, Paired Rows: 70126, Time: 2019-11-28 14:34:41.817072
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 76833, Time: 2019-11-28 14:36:05.589927
Total Rows Read: 1200000, Paired Rows: 84398, Time: 2019-11-28 14:37:20.928060
Total Rows Read: 1300000, Paired Rows: 92389, Time

Total Rows Read: 10300000, Paired Rows: 736037, Time: 2019-11-28 16:34:18.233771
Total Rows Read: 10400000, Paired Rows: 742734, Time: 2019-11-28 16:35:26.366013
Total Rows Read: 10500000, Paired Rows: 748895, Time: 2019-11-28 16:36:35.383712
Total Rows Read: 10600000, Paired Rows: 754786, Time: 2019-11-28 16:37:54.900113
Total Rows Read: 10700000, Paired Rows: 760920, Time: 2019-11-28 16:39:18.108520
Total Rows Read: 10800000, Paired Rows: 767305, Time: 2019-11-28 16:40:40.429059
Total Rows Read: 10900000, Paired Rows: 773602, Time: 2019-11-28 16:41:48.362768
Total Rows Read: 11000000, Paired Rows: 780210, Time: 2019-11-28 16:42:50.420236
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 784907, Time: 2019-11-28 16:44:31.511275
Total Rows Read: 11200000, Paired Rows: 790873, Time: 2019-11-28 16:45:32.631626
Total Rows Read: 11300000, Paired Rows: 797364, Time: 2019-11-28 16:46:36.678100
Total Rows Read: 11400000, Paired Rows: 804256, Time: 2019-11-28 16:48:06.286497
Total Rows Read:

Total Rows Read: 20200000, Paired Rows: 1404723, Time: 2019-11-28 19:13:16.639013
Total Rows Read: 20300000, Paired Rows: 1411953, Time: 2019-11-28 19:14:34.420053
Total Rows Read: 20400000, Paired Rows: 1419427, Time: 2019-11-28 19:15:47.258473
Total Rows Read: 20500000, Paired Rows: 1426498, Time: 2019-11-28 19:16:57.925449
Total Rows Read: 20600000, Paired Rows: 1433555, Time: 2019-11-28 19:18:05.895034
Total Rows Read: 20700000, Paired Rows: 1440859, Time: 2019-11-28 19:19:11.863946
Total Rows Read: 20800000, Paired Rows: 1447483, Time: 2019-11-28 19:20:19.169773
Total Rows Read: 20900000, Paired Rows: 1454199, Time: 2019-11-28 19:21:28.584357
Total Rows Read: 21000000, Paired Rows: 1460530, Time: 2019-11-28 19:23:05.571621
Cleanin up!
Total Rows Read: 21100000, Paired Rows: 1464830, Time: 2019-11-28 19:25:39.312064
Total Rows Read: 21200000, Paired Rows: 1470253, Time: 2019-11-28 19:26:59.510678
Total Rows Read: 21300000, Paired Rows: 1476395, Time: 2019-11-28 19:28:23.854355
Tota

Total Rows Read: 30100000, Paired Rows: 2076220, Time: 2019-11-28 21:48:14.682590
Total Rows Read: 30200000, Paired Rows: 2082676, Time: 2019-11-28 21:49:13.894412
Total Rows Read: 30300000, Paired Rows: 2089852, Time: 2019-11-28 21:50:16.530351
Total Rows Read: 30400000, Paired Rows: 2097293, Time: 2019-11-28 21:51:19.519604
Total Rows Read: 30500000, Paired Rows: 2104305, Time: 2019-11-28 21:52:23.990196
Total Rows Read: 30600000, Paired Rows: 2111550, Time: 2019-11-28 21:53:27.961720
Total Rows Read: 30700000, Paired Rows: 2118253, Time: 2019-11-28 21:54:29.553930
Total Rows Read: 30800000, Paired Rows: 2125825, Time: 2019-11-28 21:55:38.250149
Total Rows Read: 30900000, Paired Rows: 2133442, Time: 2019-11-28 21:57:29.609738
Total Rows Read: 31000000, Paired Rows: 2140776, Time: 2019-11-28 21:59:16.412467
Cleanin up!
Total Rows Read: 31100000, Paired Rows: 2146651, Time: 2019-11-28 22:03:39.825241
Total Rows Read: 31200000, Paired Rows: 2153417, Time: 2019-11-28 22:04:55.240226
Tota

Total Rows Read: 40000000, Paired Rows: 2749760, Time: 2019-11-29 00:37:34.173971
Cleanin up!
Total Rows Read: 40100000, Paired Rows: 2755891, Time: 2019-11-29 00:40:58.178837
Total Rows Read: 40200000, Paired Rows: 2762713, Time: 2019-11-29 00:42:18.772662
Total Rows Read: 40300000, Paired Rows: 2769949, Time: 2019-11-29 00:43:38.320151
Total Rows Read: 40400000, Paired Rows: 2777527, Time: 2019-11-29 00:44:58.749287
Total Rows Read: 40500000, Paired Rows: 2784414, Time: 2019-11-29 00:47:19.059988
Total Rows Read: 40600000, Paired Rows: 2790501, Time: 2019-11-29 00:49:47.750842
Total Rows Read: 40700000, Paired Rows: 2796598, Time: 2019-11-29 00:51:08.053056
Total Rows Read: 40800000, Paired Rows: 2803166, Time: 2019-11-29 00:52:27.297117
Total Rows Read: 40900000, Paired Rows: 2810253, Time: 2019-11-29 00:53:45.686692
Total Rows Read: 41000000, Paired Rows: 2817508, Time: 2019-11-29 00:55:03.906297
Cleanin up!
Total Rows Read: 41100000, Paired Rows: 2822923, Time: 2019-11-29 00:59:17

KeyboardInterrupt: 